# Gradient descent for MR, PET and CT
This demonstration shows how to do image reconstruction using gradient descent for different modalities. 

It builds on the the notebook *acquisition_model_mr_pet_ct.ipynb*. The first part of the notebook which creates acquisition models and simulates data from the brainweb is the same code but with fewer comments. If anything is unclear, then please refer to the other notebook to get some further information.

This demo is a jupyter notebook, i.e. intended to be run step by step.
You could export it as a Python file and run it one go, but that might
make little sense as the figures are not labelled.


Author: Christoph Kolbitsch, Edoardo Pasca
First version: 23rd of April 2021  

CCP PETMR Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2019 University College London.   
Copyright 2021 Physikalisch-Technische Bundesanstalt.

This is software developed for the Collaborative Computational
Project in Positron Emission Tomography and Magnetic Resonance imaging
(http://www.ccppetmr.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Initial set-up

In [ ]:
# Make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
# Make sure everything is installed that we need
!pip install brainweb nibabel --user

In [ ]:
# Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt

import os
import sys
import shutil
import brainweb
from tqdm.auto import tqdm

# Import MR, PET and CT functionality
import sirf.Gadgetron as mr
import sirf.STIR as pet
import cil.framework as ct

from sirf.Utilities import examples_data_path
from cil.plugins.astra.operators import ProjectionOperator as ap

# Utilities

In [ ]:
# First define some handy function definitions
# To make subsequent code cleaner, we have a few functions here. You can ignore
# ignore them when you first see this demo.

def plot_2d_image(idx,vol,title,clims=None,cmap="viridis"):
    """Customized version of subplot to plot 2D image"""
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar()
    plt.title(title)
    plt.axis("off")

def crop_and_fill(templ_im, vol):
    """Crop volumetric image data and replace image content in template image object"""
    # Get size of template image and crop
    idim_orig = templ_im.as_array().shape
    idim = (1,)*(3-len(idim_orig)) + idim_orig
    offset = (numpy.array(vol.shape) - numpy.array(idim)) // 2
    vol = vol[offset[0]:offset[0]+idim[0], offset[1]:offset[1]+idim[1], offset[2]:offset[2]+idim[2]]
    
    # Make a copy of the template to ensure we do not overwrite it
    templ_im_out = templ_im.copy()
    
    # Fill image content 
    templ_im_out.fill(numpy.reshape(vol, idim_orig))
    return(templ_im_out)

# Get brainweb data

We will download and use data from the brainweb. We will use a FDG image for PET and the PET uMAP for CT. MR usually provides qualitative images with an image contrast proportional to difference in T1, T2 or T2* depending on the sequence parameters. Nevertheless, we will make our life easy, by directly using the T1 map provided by the brainweb for MR.

In [ ]:
fname, url= sorted(brainweb.utils.LINKS.items())[0]
files = brainweb.get_file(fname, url, ".")
data = brainweb.load_file(fname)

brainweb.seed(1337)

In [ ]:
for f in tqdm([fname], desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(f, petNoise=1, t1Noise=0.75, t2Noise=0.75, petSigma=1, t1Sigma=1, t2Sigma=1)

In [ ]:
FDG_arr  = vol['PET']
T1_arr   = vol['T1']
uMap_arr = vol['uMap']

In [ ]:
# Display it
plt.figure();
slice_show = FDG_arr.shape[0]//2
plot_2d_image([1,3,1], FDG_arr[slice_show, 100:-100, 100:-100], 'FDG', cmap="hot")
plot_2d_image([1,3,2], T1_arr[slice_show, 100:-100, 100:-100], 'T1', cmap="Greys_r")
plot_2d_image([1,3,3], uMap_arr[slice_show, 100:-100, 100:-100], 'uMap', cmap="bone")

# Acquisition Models

Here we will set up the acquisition models for __MR__, __PET__ and __CT__.

## MR

In [ ]:
# 1. create MR AcquisitionData
mr_acq = mr.AcquisitionData(examples_data_path('MR') + '/grappa2_1rep.h5')

In [ ]:
# 2. calculate CSM
preprocessed_data = mr.preprocess_acquisition_data(mr_acq)

csm = mr.CoilSensitivityData()
csm.smoothness = 50
csm.calculate(preprocessed_data)

In [ ]:
# 3. calculate image template
recon = mr.FullySampledReconstructor()
recon.set_input(preprocessed_data)
recon.process()
im_mr = recon.get_output()

In [ ]:
# 4. create AcquisitionModel
acq_mod_mr = mr.AcquisitionModel(preprocessed_data, im_mr)

# Supply csm to the acquisition model 
acq_mod_mr.set_coil_sensitivity_maps(csm)

## PET

In [ ]:
# 1. create PET AcquisitionData
templ_sino = pet.AcquisitionData(examples_data_path('PET') + "/thorax_single_slice/template_sinogram.hs")

In [ ]:
# 2. create a template PET ImageData
im_pet = pet.ImageData(templ_sino)

In [ ]:
# 3. create AcquisitionModel

# create PET acquisition model
acq_mod_pet = pet.AcquisitionModelUsingRayTracingMatrix()
acq_mod_pet.set_up(templ_sino, im_pet)

## CT

In [ ]:
# 1. define AcquisitionGeometry
angles = numpy.linspace(0, 360, 50, True, dtype=numpy.float32)
ag2d = ct.AcquisitionGeometry.create_Cone2D((0,-1000), (0, 500))\
          .set_panel(128,pixel_size=3.104)\
          .set_angles(angles)

In [ ]:
# 2. get ImageGeometry
ct_ig = ag2d.get_ImageGeometry()

In [ ]:
# 3. create ImageData
im_ct = ct_ig.allocate(None)

In [ ]:
# 4. create AcquisitionModel
acq_mod_ct = ap(ct_ig, ag2d, device='cpu')

# Apply acquisition models

Here we will use the acquisition models to create simulated raw data and then do a simple reconstruction to have some initial images (i.e. starting point) for our gradient descent algorithms. For each modality we will:

 * Fill an image template (`im_mr`, `im_pet`, `im_ct`)
 * Create raw data (`raw_mr`, `raw_pet`, `raw_ct`)
 * Reconstruct an initial guess of our image using `backward`/`adjoint`

In [ ]:
# MR
im_mr = crop_and_fill(im_mr, T1_arr)
raw_mr = acq_mod_mr.forward(im_mr)
bwd_mr = acq_mod_mr.backward(raw_mr)

# PET
im_pet = crop_and_fill(im_pet, FDG_arr)
raw_pet = acq_mod_pet.forward(im_pet)
bwd_pet = acq_mod_pet.backward(raw_pet)

# CT
im_ct = crop_and_fill(im_ct, uMap_arr)
raw_ct = acq_mod_ct.direct(im_ct)
bwd_ct = acq_mod_ct.adjoint(raw_ct)

# Gradient descent or ascent depending

There are basically two things we need to be able to run a gradient descent algorithm. First we need an objective function (`obj_func`) which calculates the difference between the acquired raw data and our current image estimate. Second, we need to know the gradient of the objective function (`obj_func_grad`), because we need to know how we have to update our current image estimate in order to decrease the value of the objective function.

Both `obj_func` and `obj_func_grad` are modality specific and so here we will go through all modalities and define them. Let's start with __PET__ .

## PET

The noise in __PET__ follows a Poisson distribution and hence we can use a Poisson log-likelihood as our objective function. Luckily enough this is already part of __SIRF__ and hence we can simply create the objective function by providing the raw __PET__ data and __PET__ image data.

In [ ]:
obj_fun_pet_sirf = pet.make_Poisson_loglikelihood(raw_pet)
obj_fun_pet_sirf.set_up(bwd_pet)

Because `obj_func_pet` cannot be called directly but `obj_func_pet.value()` has to be used, we write a quick wrapper around it

In [ ]:
def obj_fun_pet(curr_image_estimate):
    return(obj_fun_pet_sirf.value(curr_image_estimate))

The gradient is also already implemented and can simply be calculated by calling `obj_fun_pet.gradient`. Nevertheless, to be more explicit and consistent with the other modalities we will define a new function to do the job:

In [ ]:
def obj_fun_grad_pet(curr_image_estimate):
    # The 0 here means, only the gradient for subset 0 is returned. 
    # We will just accept this as is here, because subsets are too advanced for this demo here.
    return(obj_fun_pet_sirf.gradient(curr_image_estimate, 0))

In __PET__ we need to make sure that all the image values are positive, so we will create a small function for this

In [ ]:
def make_positive(image_array):
    """truncate any negatives to zero"""
    image_array[image_array<0] = 0
    return image_array

Great, __PET__ is all done, now we will continue with __CT__ .

## CT

--> CT CODE HERE

## MR

And last but not least __MR__ :

In [ ]:
def obj_fun_mr(curr_image_estimate):
    c =  acq_mod_mr.forward(curr_image_estimate) - raw_mr
    return(0.5 * c.norm() ** 2)

In [ ]:
def obj_fun_grad_mr(curr_image_estimation):
    # We are returning the negative gradient, because we are going to add it later to our image estimate
    return(-acq_mod_mr.backward(acq_mod_mr.forward(curr_image_estimate) - raw_mr))

Now we have all our `obj_func` and `obj_func_grad` we will select one modality and then implement the gradient descent/ascent appproach. We also need an image `init_image` to start with. Here we will simply use the simple reconstruction which did above.

In [ ]:
obj_fun = obj_fun_pet
obj_fun_grad = obj_fun_grad_pet
init_image = bwd_pet

First we need to define the number of iterations for the gradient descent algorithm

In [ ]:
# Number of iterations
num_iters = 32

Now we come to the probably most important paramter for gradient descent, the infamous __step-size__ . Unfortunately, the gradient only gives us the direction along which we need to update the image, but does not tell us by how much we have to go in this direction. Therefore, we need to define how far we step along the gradient direction in each iteration by hand. 

To make sure the step-size is adapted to each modality as much as possible, we won't define the step-size directly, but we will calculate the step-size as `tau` times the norm of the current image estimate. Nevertheless, this is still just a guess. 

If the step-size is too small, we have very slow convergence. If the step-size is too large, then we are overstepping our target image and the objective function starts to oscillate. The value below is reasonable for __PET__, but for the other modalities you will have to optimise it to ensure good convergence. 

In [ ]:
# Relative step-size
tau = .3

Then we need to create an image as a starting point for the gradient descent algorithm and we will also create a numpy array, where we can store the image estimate at each iteration

In [ ]:
# Initial image
curr_image_estimate = init_image.clone()

# We will do the update of the image estimate in Python, so we also need the numpy array of our image data object
curr_image_estimate_array = curr_image_estimate.as_array()

# Array for images at each iteration
image_iteration = numpy.zeros(shape=(num_iters + 1,) + curr_image_estimate_array.shape)

# We use numpy.abs here to be compatible with MR complex data
image_iteration[0, :, :, :] = numpy.abs(curr_image_estimate_array) 

# Variable to store the current value of the objective function
obj_func_values = numpy.zeros(shape=(num_iters + 1,))
obj_func_values[0] = obj_fun(curr_image_estimate)

Now we can finally write down our gradient descent algorithm

In [ ]:
for i in range(1, num_iters+1):  
    # First we calculate the gradient to find out how we need to update our current image estimate
    grad = obj_fun_grad(curr_image_estimate)
    grad_array = grad.as_array()

    # Compute step-size relative to current image-norm
    step_size = tau * norm(curr_image_estimate_array) / norm(grad_array)

    # Perform gradient ascent step
    curr_image_estimate_array =  curr_image_estimate_array + step_size * grad_array

    # In PET we have to ensure values are positive. 
    # We check if data is complex and therefore MR, or not and therefore PET or CT
    if not curr_image_estimate_array.dtype in [numpy.complex, numpy.complex64]:
        curr_image_estimate_array = make_positive(curr_image_estimate_array)
    
    # Update the image
    curr_image_estimate.fill(curr_image_estimate_array)

    # Compute objective function value for plotting, and write some diagnostics
    obj_func_values[i] = obj_fun(curr_image_estimate)
    
    # We use numpy.abs here to be compatible with MR complex data
    image_iteration[i,:,:,:] = numpy.abs(curr_image_estimate_array)

In [ ]:
# Plot objective function values
plt.figure()
plt.title('Objective function value')
plt.plot(obj_func_values, 'o-b')

In [ ]:
# Plot a slice for different iterations
centre_slice = image_iteration.shape[1]//2
plt.figure();
for i in range(4):
    curr_it = i*8
    plot_2d_image([2,2,i+1], image_iteration[curr_it,centre_slice,:,:], 'It '+str(curr_it), cmap="viridis")